## In this notebook

- Create datasets needed for the analysis.

In [1]:
import os
import dotenv
import duckdb

from utils import UHC_PLAYERS

In [2]:
S3_BUCKET = "frozen-facts-center-prod"

OUT_FILE_PATH_SHOTS = "/usr/src/app/data/shots.parquet"
OUT_FILE_PATH_PENALTIES = "/usr/src/app/data/penalties.parquet"
OUT_FILE_PATH_PLAYERS = "/usr/src/app/data/players.parquet"

## Create DB connection

In [3]:
def get_connection() -> duckdb.duckdb.DuckDBPyConnection:
    dotenv.load_dotenv()
        
    return (
        duckdb
        .connect(database=":memory:", read_only=False)
        .execute(
            "SET s3_region='{}'; SET s3_access_key_id='{}'; SET s3_secret_access_key='{}';".format(
                os.environ.get("S3_REGION"),
                os.environ.get("S3_ACCESS_KEY_ID"),
                os.environ.get("S3_SECRET_ACCESS_KEY"),
            )
        )
    )

con = get_connection()

## Create datasets

Load data from S3 and save it locally.

In [4]:
def create_dataset(query: str, out_file_path: str) -> None:
    df = con.execute(query).df()
    print(f"- pulled {len(df):,.0f} rows")
    
    df.to_parquet(out_file_path)
    print(f"- saved data into {out_file_path}")

### Shots

In [5]:
query = f"""
    select
        game_id,
        game_date,
        away_team_id,
        home_team_id,
        period,
        period_type,
        time_in_period,
        situation_code,
        home_team_defending_side,
        event_type,
        sort_order,
        x_coord,
        y_coord,
        zone_code,
        event_owner_team_id,
        shot_type,
        shooting_player_id,
        goalie_in_net_id
    from read_parquet("s3://{S3_BUCKET}/base_shots.parquet")
"""


create_dataset(query=query, out_file_path=OUT_FILE_PATH_SHOTS)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

- pulled 600,557 rows
- saved data into /usr/src/app/data/shots.parquet


### Penalties

In [6]:
query = f"""
    select
        game_id, 
        away_team_id, 
        home_team_id, 
        sort_order,
        event_owner_team_id,
        ifnull(served_by_player_id, committed_by_player_id) as served_by_player_id,
        period,
        period_type,
        time_in_period,
        duration
    from read_parquet("s3://{S3_BUCKET}/base_penalties.parquet")
"""


create_dataset(query=query, out_file_path=OUT_FILE_PATH_PENALTIES)

- pulled 38,746 rows
- saved data into /usr/src/app/data/penalties.parquet


### Utah HC players

In [7]:
query = f"""
with 

players as (
    select * from read_parquet("s3://{S3_BUCKET}/stg_players_general_info.parquet")
),

teams as (
    select * from read_parquet("s3://{S3_BUCKET}/dim_teams.parquet")
)

select 
    player_id, 
    full_name, 
    position_code,
    season_long, 
    season_type, 
    team_full_name, 
    games_played 

from players

left join teams
  on teams.id = players.team_id

where players.player_id in ({','.join(str(player_id) for player_id in UHC_PLAYERS)})

order by player_id, season_long, season_type
"""


create_dataset(query=query, out_file_path=OUT_FILE_PATH_PLAYERS)

- pulled 93 rows
- saved data into /usr/src/app/data/players.parquet


## Results

- Datasets created.